# Imports

In [166]:
import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import re
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, f1_score
import xgboost as xgb
import torch.nn as nn
import torch.optim as optim
import copy
import torch.multiprocessing
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt

torch.manual_seed(0)
torch.multiprocessing.set_sharing_strategy('file_system')

# FFNN Declaration

In [252]:
class FFNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FFNN, self).__init__()
        
        self.fcr = nn.Linear(input_dim['RNA'], 100)
        self.fcc = nn.Linear(input_dim['CNV'], 100)
        
        self.fcj = nn.Linear(200, 40)
        
        self.bn1 = nn.BatchNorm1d(200)
        self.bn2 = nn.BatchNorm1d(40)
        
        self.fc = nn.Linear(40,3)

        
    def forward(self, x1, x2, test=False):
        
        out1 = F.dropout(x1, 0.4, training=self.training)
        out1 = torch.tanh(self.fcr(out1))
        
        out2 = F.dropout(x2, 0.4, training=self.training)
        out2 = torch.tanh(self.fcc(out2))
        
        x = torch.cat([out1,out2], axis=1)
        x = self.bn1(x)
        x = F.dropout(x, 0.5, training=self.training)
        x = self.fcj(x)
        x = self.bn2(x)
        
        x = self.fc(x)
        
        return x

class FFNNDataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, inputs, labels):
        'Initialization'
        self.inputs_rna = inputs['RNA']
        self.inputs_cnv = inputs['CNV']
        self.labels = labels

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.inputs_rna)

    def __getitem__(self, index):
        'Generates one sample of data'
        x_rna = torch.from_numpy(self.inputs_rna[index]).float()
        x_cnv = torch.from_numpy(self.inputs_cnv[index]).float()
        label = torch.from_numpy(self.labels[index]).float()
        return x_rna, x_cnv, label



In [7]:
all_rna = pd.read_csv('../RNA-Seq/RNA-ExpAll-LC.csv.gz', compression='gzip')
all_cnv = pd.read_csv('../Copy-Number-Variation/gene_matrix_CNV_noNA.csv.gz', compression='gzip')

/home/franciscocp/multiomic-classification-NSCLC/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,2

In [20]:
classes = np.array(['healthy', 'adeno', 'squa'])

ohe = preprocessing.OneHotEncoder(sparse=False)
ohe.fit(classes.reshape(-1,1))

OneHotEncoder(sparse=False)

In [33]:
def get_val_set(x, y, classes, percentage = 0.1):
    np.random.seed(42)  
    x_train = np.array([]).reshape(0,x.shape[1])
    y_train = np.array([]).reshape(0,y.shape[1])
    x_val = np.array([]).reshape(0,x.shape[1])
    y_val = np.array([]).reshape(0,y.shape[1])
    for c in classes:
        indexes = np.where(y.argmax(axis=1) == c)[0]
        np.random.shuffle(indexes)
        len_val = int(percentage * len(indexes))
        len_train = len(indexes) - len_val
        index_train = indexes[0:len_train]
        index_val = indexes[len_train:]
        x_train = np.concatenate([x_train, x[index_train,...]], axis=0)
        y_train = np.concatenate([y_train, y[index_train]], axis=0)
        x_val = np.concatenate([x_val, x[index_val,...]], axis=0)
        y_val = np.concatenate([y_val, y[index_val]], axis=0)
    
    index_train = list(range(x_train.shape[0]))
    index_val = list(range(x_val.shape[0]))
    np.random.shuffle(index_train)
    np.random.shuffle(index_val)
    
    return x_train[index_train,...],y_train[index_train], x_val[index_val,...], y_val[index_val]

def weights_init(m):
    # Xavier weight initialization
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)
        m.bias.data.fill_(0.01)

In [269]:
splits = 10
path = '../Copy-Number-Variation/Splits_10CV/'

save_xlsx = False
accs = []
test_preds_all = []
test_labels_all = []

if save_xlsx:
    writer = pd.ExcelWriter('early_integration/intermediate_integration_test.xlsx', engine='openpyxl')
    
for split in tqdm(range(splits)):
    name_cnv = '../Copy-Number-Variation/p-valueDEGsAll/DEGs_CNV_train'+str(split)+'_p0-001_m0-1_cov3.csv'
    name_rna = '../RNA-Seq/limma-degs/limmadegs_RNA_train'+str(split)+'_lfc1-5_p0-001_cov2.csv'
    
    # RNA
    degs_rna = pd.read_csv(name_rna)
    columns = np.concatenate([['Case_IDs'], degs_rna['x'].values], axis=0)
    df_rna = all_rna[columns]
    
    # CNV
    df_cnv = pd.read_csv(name_cnv)
    df_cnv = df_cnv.rename({'Unnamed: 0': 'Case_IDs'}, axis=1)
    
    equal_case_ids = list(set(df_cnv['Case_IDs'].values) & set(df_rna['Case_IDs'].values))
    df_rna = df_rna.loc[df_rna['Case_IDs'].isin(equal_case_ids)]
    df_cnv = df_cnv.loc[df_cnv['Case_IDs'].isin(equal_case_ids)]
    assert df_rna.shape[0] == df_cnv.shape[0]
        
    train_f = open(path+'train_'+str(split)+'.txt', 'r')
    train_caseids = train_f.readlines()
    train_f.close()
    val_f = open(path+'val_'+str(split)+'.txt', 'r')
    val_caseids = val_f.readlines()
    val_f.close()

    train_cids = []
    for cid in train_caseids:
        train_cids.append(cid.replace('\n', ''))

    val_cids = []
    for cid in val_caseids:
        val_cids.append(cid.replace('\n', '')) 

    train_final = []
    for i in range(len(list(df_rna['Case_IDs'].values))):
        resu = re.match('|'.join(train_cids),list(df_rna['Case_IDs'].values)[i])
        if resu:
            if resu.group(0) != '':
                train_final.append(i)

    val_final = []
    for j in range(len(list(df_rna['Case_IDs'].values))):
        resu = re.match('|'.join(val_cids),list(df_rna['Case_IDs'].values)[j])
        if resu:
            if resu.group(0) != '':
                val_final.append(j)

    df_train_rna = df_rna.iloc[train_final,]
    df_test_rna = df_rna.iloc[val_final,]
    
    df_train_cnv = df_cnv.iloc[train_final,]
    df_test_cnv = df_cnv.iloc[val_final,]
    
    case_ids_test = df_test_rna['Case_IDs']
    y_test = all_rna['labelsAll'].loc[all_rna['Case_IDs'].isin(case_ids_test)].values
    y_test = np.where(y_test == 'Healthy', 'healthy', y_test)
    y_test = np.where(y_test == 'Adenocarcinoma', 'adeno', y_test)
    y_test = np.where(y_test == 'Squamous', 'squa', y_test)
    
    case_ids_train = df_train_rna['Case_IDs']
    y_train = all_rna['labelsAll'].loc[all_rna['Case_IDs'].isin(case_ids_train)].values
    y_train = np.where(y_train == 'Healthy', 'healthy', y_train)
    y_train = np.where(y_train == 'Adenocarcinoma', 'adeno', y_train)
    y_train = np.where(y_train == 'Squamous', 'squa', y_train)
     
    y_train_ohe = ohe.transform(y_train.reshape(-1,1))
    y_test_ohe = ohe.transform(y_test.reshape(-1,1))
    
    x_train_rna = df_train_rna.iloc[:, 1:].values
    x_test_rna = df_test_rna.iloc[:, 1:].values
    
    x_train_cnv = df_train_cnv.iloc[:, 1:].values
    x_test_cnv = df_test_cnv.iloc[:, 1:].values
    
    print('End data read...')
    classes_ = [0,1,2]
    x_train_rna_new, y_train_ohe_new, x_val_rna, y_val_ohe = get_val_set(x_train_rna, y_train_ohe, classes_, percentage = 0.1)
    x_train_cnv_new, _, x_val_cnv, _ = get_val_set(x_train_cnv, y_train_ohe, classes_, percentage = 0.1)
    
    # FFNN
    
    scaler_rna = StandardScaler()
    scaler_cnv = StandardScaler()
    x_train_rna_new = scaler_rna.fit_transform(x_train_rna_new)
    x_val_rna = scaler_rna.transform(x_val_rna)
    x_test_rna = scaler_rna.transform(x_test_rna)
    
    x_train_cnv_new = scaler_cnv.fit_transform(x_train_cnv_new)
    x_val_cnv = scaler_cnv.transform(x_val_cnv)
    x_test_cnv = scaler_cnv.transform(x_test_cnv)
    
    train = {'RNA': x_train_rna_new, 'CNV': x_train_cnv_new}
    val = {'RNA': x_val_rna, 'CNV': x_val_cnv}
    test = {'RNA': x_test_rna, 'CNV': x_test_cnv}
    
    dataset_train = FFNNDataset(train, y_train_ohe_new)
    dataset_val = FFNNDataset(val, y_val_ohe)
    dataset_test = FFNNDataset(test, y_test_ohe)

    dataloader_train = DataLoader(dataset_train, batch_size=64,
                                  shuffle=True, num_workers=4,
                                  pin_memory=False)
    dataloader_val = DataLoader(dataset_val, batch_size=1,
                                  shuffle=True, num_workers=4,
                                  pin_memory=False)
    dataloader_test = DataLoader(dataset_test, batch_size=1,
                                  shuffle=False, num_workers=4,
                                  pin_memory=False)
    
    input_dim = {
        'RNA': x_train_rna.shape[1],
        'CNV': x_train_cnv.shape[1]
    }

    output_dim = 3

    model = FFNN(input_dim, output_dim)

    model.apply(weights_init)
    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=0)
    losses = []
    num_epochs = 15
    dataloaders = {
        'train': dataloader_train,
        'val': dataloader_val
    }
    
    # Start training
    best_epoch = 0
    best_acc = 100
    val_preds = []
    val_labels = []
    val_labels_auc = []
    val_preds_auc = []
    train_preds = []
    train_labels = []
    train_preds_auc = []
    train_labels_auc = []
    best_model_wts = copy.deepcopy(model.state_dict())
    losses = {
        'train': [],
        'val': []
    }
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        batch_loss = 0
        sizes = {}
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            sizes[phase] = 0
            running_loss = 0
            running_corrects = 0
            for inputs_rna,inputs_cnv, labels in dataloaders[phase]:
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs_rna,inputs_cnv)
                    _, preds = torch.max(outputs, 1)
                    _, mlabels = torch.max(labels, 1)
                    
                    loss = criterion(outputs, mlabels)
                if phase == 'val':
                    val_preds += list(preds.numpy())
                    val_labels += list(mlabels.numpy())
                    val_labels_auc += [labels.numpy()]
                    val_preds_auc += [preds.numpy()]

                
                # Accumulating the loss over time
                running_loss += loss.item() * inputs_rna.size(0)
                running_corrects += torch.sum(preds == mlabels)
                sizes[phase] += inputs_rna.size(0)
                
                # backward + optimize only if in training phase
                if phase == 'train':
                    train_preds += list(preds.numpy())
                    train_labels += list(mlabels.numpy())
                    train_labels_auc += [labels.numpy()]
                    train_preds_auc += [preds.numpy()]
                    # Getting gradients w.r.t. parameters
                    loss.backward()

                    # Updating parameters
                    optimizer.step()
                
            epoch_loss = running_loss / sizes[phase]
            epoch_acc = running_corrects.item() / sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc))
            losses[phase].append(epoch_loss)
            if phase == 'val' and epoch_loss < best_acc:
                best_epoch = epoch
                best_acc = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            
                    
    print('Best epoch {}'.format(best_epoch))
    model.load_state_dict(best_model_wts)
    
    plt.figure()
    plt.plot(list(range(num_epochs)),losses['train'], label='train', color='blue')
    plt.plot(list(range(num_epochs)),losses['val'], label='val', color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.close()
    
    # Test in validation set
    
    sizes = 0
        
    model.eval()
    running_corrects = 0
    running_loss = 0
    test_preds = []
    test_labels = []
    test_labels_auc = []
    test_preds_auc = []
    test_probs_luad = []
    test_probs_hlt = []
    test_probs_lusc = []
    phase = 'test'
    for inputs_rna,inputs_cnv, labels in dataloader_test:
        optimizer.zero_grad()
        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs_rna,inputs_cnv, test=True)
            probs = F.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            _, mlabels = torch.max(labels, 1)

            loss = criterion(outputs, mlabels)
        # Accumulating the loss over time
        running_loss += loss.item() * inputs_rna.size(0)
        running_corrects += torch.sum(preds == mlabels)
        sizes += inputs_rna.size(0)
        test_preds += list(preds.cpu().numpy())
        test_labels += list(mlabels.numpy())
        test_probs = probs.numpy()[0]
        test_probs_luad += [test_probs[0]]
        test_probs_hlt += [test_probs[1]]
        test_probs_lusc += [test_probs[2]]
        test_labels_auc += [labels.numpy()]
        test_preds_auc += [preds.numpy()]
    epoch_loss = running_loss / sizes
    epoch_acc = running_corrects.item() / sizes
    print('Test Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    accs.append(epoch_acc)
    test_preds_all.append(test_preds)
    test_labels_all.append(test_labels)
    
    if save_xlsx:
        data_test = pd.DataFrame()
        data_test['Case IDs'] = case_ids_test
        data_test['Intregation Prob LUAD'] = test_probs_luad
        data_test['Integration Prob HLT'] = test_probs_hlt
        data_test['Integration Prob LUSC'] = test_probs_lusc
        data_test['Integration Pred'] = test_preds
        data_test['Real'] = y_test_ohe.argmax(axis=1)
        data_test.to_excel(writer, sheet_name='split_'+str(split), index=False)

if save_xlsx:
    writer.close()

  0%|          | 0/10 [00:00<?, ?it/s]

> <ipython-input-269-580fd088873a>(99)<module>()
     97 
     98     import pdb; pdb.set_trace()
---> 99     print('End data read...')
    100     classes_ = [0,1,2]
    101     x_train_rna_new, y_train_ohe_new, x_val_rna, y_val_ohe = get_val_set(x_train_rna, y_train_ohe, classes_, percentage = 0.1)

ipdb> x_train_rna.shape
(812, 371)
ipdb> x_train_cnv.shape
(812, 1798)
ipdb> exit


BdbQuit: 